import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
# Imports

Things to add to this notebook
- bar chart showing difference between total num of restaurants and num of restaurants that match
- the "uniqueness" of the city's data 

## Libraries

In [2]:
# import geopandas
import random
import warnings
warnings.filterwarnings('ignore')
from functions import *

## Data

In [ ]:
original_df = pd.read_csv("../my_saved_data/all_correct_restaurants.csv", index_col=0)
# original_df.shape

In [ ]:
rest_df = pd.read_csv("../my_saved_data/all_correct_restaurants_and_regions.csv", index_col=0)
# 282 more restaurants in this version than last one! 
# rest_df.shape[0]

In [ ]:
with open("../my_saved_data/demonym_dictionary.pickle", "rb+") as f:
    demonym_dict = pickle.load(f)

### Checking differences between the original data set and the new data set 
(March 2)

In [ ]:
# # how many restaurants in total 
# print(f"The original dataset had {rest_df.shape[0] - original_df.shape[0]} fewer matches than this one")

In [ ]:
# original_dataset_groupby = original_df.groupby("City").count()
# rest_df_groupby = rest_df.groupby("City").count()

In [ ]:
# original_df_values = original_dataset_groupby['Match'].values
# rest_df_values = rest_df_groupby['Restaurant'].values

In [ ]:
# value_differences = rest_df_values - original_df_values
# for idx, item in enumerate(value_differences):
#     print(f"{original_dataset_groupby.index[idx]} had {item} more values ({round(item / original_df_values[idx], 2) * 100}%) in the new iteration of this project than the old one.")

# Data Collection

## Count of Restaurants for each city

In [ ]:
rest_count_dict = {}
rest_count_dict['Chicago'] = len(rest_df[rest_df['City'] == 'Chi'])
rest_count_dict['Dallas'] = len(rest_df[rest_df['City'] == 'Dal'])
rest_count_dict['NYC'] = len(rest_df[rest_df['City'] == 'NYC'])
rest_count_dict['SF'] = len(rest_df[rest_df['City'] == 'SF'])

In [ ]:
# chicago has many more because smaller types of businesses were also counted (paleterias, food trucks, etc.)
rest_count_dict

## Count of Regions for Each City

In [ ]:
nyc_regions = count_regions_in_city("NYC", rest_df)
dal_regions = count_regions_in_city("Dal", rest_df)
sf_regions = count_regions_in_city("SF", rest_df)
chi_regions = count_regions_in_city("Chi", rest_df)

## Percent of restaurants representing a region for each city

In [ ]:
regions = list(demonym_dict.keys())

In [ ]:
pct_df = pd.DataFrame(data = regions, columns = ['Region'])

pct_df.sort_values(by = 'Region', inplace=True)
pct_df.reset_index(inplace=True, drop=True)

In [ ]:
pct_df['NYC'] = add_regions_for_city_regions_count(nyc_regions, regions)
pct_df['Chi'] = add_regions_for_city_regions_count(chi_regions, regions)
pct_df['Dal'] = add_regions_for_city_regions_count(dal_regions, regions)
pct_df['SF'] = add_regions_for_city_regions_count(sf_regions, regions)

In [ ]:
pct_df['NYC_pct'] = round(pct_df['NYC'] / sum(list(nyc_regions.values())), 2)
pct_df['Chi_pct'] = round(pct_df['Chi'] / sum(list(chi_regions.values())), 2)
pct_df['Dal_pct'] = round(pct_df['Dal'] / sum(list(dal_regions.values())), 2)
pct_df['SF_pct'] = round(pct_df['SF'] / sum(list(sf_regions.values())), 2)

## Most prevalent regions for each city

In [ ]:
pct_df.sort_values(by=['NYC_pct'], ascending=False)[0:5]

In [ ]:
pct_df.sort_values(by=['Chi_pct'], ascending=False)[0:5]

In [ ]:
pct_df.sort_values(by=['Dal_pct'], ascending=False)[0:5]

In [ ]:
pct_df.sort_values(by=['SF_pct'], ascending=False)[0:5]

## Regions That are Over 20% of a city's mex restaurants 

In [ ]:
over_20_regions = []
for idx, x in enumerate(pct_df['Region']):
    for col in pct_df.columns[-4:]:
        if pct_df[col][idx] >= 0.20:
            if x not in over_20_regions:
                over_20_regions.append(x)
over_20_regions

In [ ]:
over_20_df = pct_df[pct_df['Region'].isin(over_20_regions)][['Region',"Chi_pct", "Dal_pct", 'NYC_pct', "SF_pct"]]

## Saving Data 

In [ ]:
# pct_df.to_csv("../my_saved_data/new_corrected_pct_df.csv")
# over_20_df.to_csv("../my_saved_data/regions_over_20_pct_of_city.csv")

# Culinary Regions of Mexico

## Adding Culinary Regions to DataFrame

In [ ]:
region_to_cul_region = {}
for key, value in cul_reg_dict.items():
    for item in value:
        region_to_cul_region[item] = key

In [ ]:
rest_df['Culinary_region'] = rest_df['Final_region'].replace(region_to_cul_region)

In [ ]:
rest_df

## Grouping Culinary Regions Together

In [ ]:
orig_super_region_df = rest_df.groupby('Culinary_region').count()

In [ ]:
super_region_df = orig_super_region_df[['Restaurant']]
super_region_df['Percentage'] = round(super_region_df['Restaurant'] / rest_df.shape[0], 2)

In [ ]:
super_region_df

## Grouping by Cities and Culinary Regions

In [ ]:
nyc_regions = count_regions_in_city("NYC", rest_df)
dal_regions = count_regions_in_city("Dal", rest_df)
sf_regions = count_regions_in_city("SF", rest_df)
chi_regions = count_regions_in_city("Chi", rest_df)

In [ ]:
nyc_regions

In [ ]:
sum(nyc_regions.values())

In [ ]:
cities_cul_regions_list = []
for item in [nyc_regions, dal_regions, sf_regions, chi_regions]:
    item_df = pd.DataFrame.from_dict(item, orient = 'index').reset_index()
    item_df.columns = ['Region', 'Count']
    item_df['Culinary_region'] = item_df['Region'].replace(region_to_cul_region)
    num_of_restaurants = sum(item.values())
    item_df['Percentage'] = item_df['Count'] / num_of_restaurants
    cities_cul_regions_list.append(item_df)
cities_cul_regions_list[0]['City'] = 'NYC'
cities_cul_regions_list[1]['City'] = 'Dal'
cities_cul_regions_list[2]['City'] = 'SF'
cities_cul_regions_list[3]['City'] = 'Chi'
orig_cities_cul_regions_df = pd.concat(cities_cul_regions_list)

In [ ]:
cities_cul_regions_df = orig_cities_cul_regions_df.groupby(["City", "Culinary_region"]).sum()
cities_cul_regions_df['Percentage'] = round(cities_cul_regions_df['Percentage'], 2)

In [ ]:
cities_cul_regions_df2 = orig_cities_cul_regions_df.groupby(["Culinary_region", "City"]).sum()
cities_cul_regions_df2['Percentage'] = round(cities_cul_regions_df2['Percentage'], 2)

In [ ]:
# can I reshape this so that the cities are in separate columns
cities_cul_regions_df2

In [ ]:
cities_cul_regions_df